In [30]:
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install sqlalchemy




  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/68/89/247a6a5bfcc5f2f27b08ce7a1fbc2ee9e54fa09ccc9fa48e8c0e08e78cbd/SQLAlchemy-2.0.21-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/192.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/192.5 kB ? eta -:--:--
     ------------- ----------------------- 71.7/192.5 kB 975.2 kB/s eta 0:00:01
     -------------------------------------- 192.5/192.5 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 25.1 MB/s eta 0:00:01
   ---------------------- ------

In [139]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine

host=os.getenv("DB_HOST")
database=os.getenv("DB_NAME")
user=os.getenv("DB_USERNAME")
password=os.getenv("DB_PASSWORD")


conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()
# Create the SQLAlchemy engine using the MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

#table_name = "stg_season_schedule"  # Name of the SQL table
#df.to_sql(table_name, engine, if_exists='replace', index=False, method='multi')

#mycursor.execute("DROP TABLE stg_nba_schedule")
#sql_query = f"SELECT * FROM {table_name}"

# Execute the SQL query and return the result as a DataFrame
#df = pd.read_sql_query(sql_query, engine)
#conn.close()

In [2]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #print(f'game_id: {gameID} team_id: {teamID} season:{seasonYear}')
        gameStats = cumestatsteam.CumeStatsTeam(game_ids='00'+str(gameID),league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        #print(gameStats)
        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    #print(data)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [7]:
sql_query = """ SELECT 
                    sss.* 
                FROM 
                    stg_season_schedule sss
                LEFT JOIN 
                    stg_game_stats sgs1 on sss.game_id = sgs1.game_id and sss.home_team_id = sgs1.team_id
                LEFT JOIN 
                    stg_game_stats sgs2 on sss.game_id = sgs2.game_id and sss.away_team_id = sgs2.team_id
                WHERE 
                    sgs1.team_id is null
                      """

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
df

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID


In [4]:
failed_attempts = 0
for _,row in df.iterrows():
    try:
        getSingleGameMetrics(row['GAME_ID'],row['HOME_TEAM_ID'],row['AWAY_TEAM_ID'],row['AWAY_TEAM_NICKNAME'],row['SEASON'],row['GAME_DATE']).to_sql('stg_game_stats', engine, if_exists='append', index=False, method='multi')
    except:
        print(f'failed attempt number {failed_attempts}')
        failed_attempts+= 1
        continue

In [34]:
# insert into table the new record
# based on existing records determine records to pull 

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID
0,05/16/2021 Rockets at Hawks,22001066,2020-21,2021-05-16,Hawks,1610612737,Rockets,1610612745
1,05/13/2021 Magic at Hawks,22001049,2020-21,2021-05-13,Hawks,1610612737,Magic,1610612753
2,05/12/2021 Wizards at Hawks,22001042,2020-21,2021-05-12,Hawks,1610612737,Wizards,1610612764
3,05/10/2021 Wizards at Hawks,22001026,2020-21,2021-05-10,Hawks,1610612737,Wizards,1610612764
4,05/06/2021 Hawks at Pacers,22001000,2020-21,2021-05-06,Pacers,1610612754,Hawks,1610612737
...,...,...,...,...,...,...,...,...
3535,10/25/2022 Pistons at Wizards,22200052,2022-23,2022-10-25,Wizards,1610612764,Pistons,1610612765
3536,03/09/2023 Hornets at Pistons,22200992,2022-23,2023-03-09,Pistons,1610612765,Hornets,1610612766
3537,02/27/2023 Pistons at Hornets,22200919,2022-23,2023-02-27,Hornets,1610612766,Pistons,1610612765
3538,02/03/2023 Hornets at Pistons,22200785,2022-23,2023-02-03,Pistons,1610612765,Hornets,1610612766


In [90]:
query = """select * from stg_nba_schedule""" 
pd.read_sql_query(query,engine)

,gameId,gameCode,gameStatus,gameStatusText,gameSequence,gameDateEst,gameTimeEst,gameDateTimeEst,gameDateUTC,gameTimeUTC,...,arenaName,arenaState,arenaCity,postponedStatus,branchLink,gameSubtype,homeTeamID,awayTeamID,homeTeamName,awayTeamName
0,0012300001,20231005/DALMIN,1,12:00 pm ET,1,2023-10-05T00:00:00Z,1900-01-01T12:00:00Z,2023-10-05T12:00:00Z,2023-10-05T04:00:00Z,1900-01-01T17:00:00Z,...,Etihad Arena,,Abu Dhabi,A,,,1610612750,1610612742,Timberwolves,Mavericks
1,0012300002,20231007/MINDAL,1,12:00 pm ET,1,2023-10-07T00:00:00Z,1900-01-01T12:00:00Z,2023-10-07T12:00:00Z,2023-10-07T04:00:00Z,1900-01-01T17:00:00Z,...,Etihad Arena,,Abu Dhabi,A,,,1610612742,1610612750,Mavericks,Timberwolves
2,0012300003,20231007/LALGSW,1,8:30 pm ET,2,2023-10-07T00:00:00Z,1900-01-01T20:30:00Z,2023-10-07T20:30:00Z,2023-10-07T04:00:00Z,1900-01-02T01:30:00Z,...,Chase Center,CA,San Francisco,A,,,1610612744,1610612747,Warriors,Lakers
3,0012300004,20231008/CHIMIL,1,1:00 pm ET,1,2023-10-08T00:00:00Z,1900-01-01T13:00:00Z,2023-10-08T13:00:00Z,2023-10-08T04:00:00Z,1900-01-01T18:00:00Z,...,Fiserv Forum,WI,Milwaukee,A,,,1610612749,1610612741,Bucks,Bulls
4,0012300005,20231008/PHXDET,1,3:00 pm ET,2,2023-10-08T00:00:00Z,1900-01-01T15:00:00Z,2023-10-08T15:00:00Z,2023-10-08T04:00:00Z,1900-01-01T20:00:00Z,...,Little Caesars Arena,MI,Detroit,A,,,1610612765,1610612756,Pistons,Suns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,0022301196,20240414/DALOKC,1,3:30 pm ET,11,2024-04-14T00:00:00Z,1900-01-01T15:30:00Z,2024-04-14T15:30:00Z,2024-04-14T04:00:00Z,1900-01-01T20:30:00Z,...,Paycom Center,OK,Oklahoma City,A,,,1610612760,1610612742,Thunder,Mavericks
1300,0022301197,20240414/DETSAS,1,3:30 pm ET,12,2024-04-14T00:00:00Z,1900-01-01T15:30:00Z,2024-04-14T15:30:00Z,2024-04-14T04:00:00Z,1900-01-01T20:30:00Z,...,Frost Bank Center,TX,San Antonio,A,,,1610612759,1610612765,Spurs,Pistons
1301,0022301198,20240414/UTAGSW,1,3:30 pm ET,13,2024-04-14T00:00:00Z,1900-01-01T15:30:00Z,2024-04-14T15:30:00Z,2024-04-14T04:00:00Z,1900-01-01T20:30:00Z,...,Chase Center,CA,San Francisco,A,,,1610612744,1610612762,Warriors,Jazz
1302,0022301199,20240414/HOULAC,1,3:30 pm ET,14,2024-04-14T00:00:00Z,1900-01-01T15:30:00Z,2024-04-14T15:30:00Z,2024-04-14T04:00:00Z,1900-01-01T20:30:00Z,...,Crypto.com Arena,CA,Los Angeles,A,,,1610612746,1610612745,Clippers,Rockets


In [22]:
from nba_api.stats.endpoints import playercareerstats
import requests

beg = 'http://'


headers = requests.utils.default_headers()

headers.update(
    {'Host': 'stats.nba.com', 
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0', 
    'Accept': 'application/json, text/plain, */*', 
    'Accept-Language': 'en-US,en;q=0.5', 
    'Accept-Encoding': 'gzip, deflate, br', 
    'x-nba-stats-origin': 'stats', 
    'x-nba-stats-token': 'true', 
    'Connection': 'keep-alive', 
    'Referer': 'https://stats.nba.com/', 
    'Pragma': 'no-cache', 
    'Cache-Control': 'no-cache'}
)
# Create the proxy URL with credentials
#proxy_url_with_credentials = f"{beg}{username}:{password}@{proxy_ip}:{proxy_port}"


career = playercareerstats.PlayerCareerStats(player_id='203999',proxy=proxy',headers=headers,timeout=600) 

In [70]:
import requests 

url = 'https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json'
response = requests.get(url)
data = response.json()['leagueSchedule']['gameDates']

In [76]:
data[0]['games'][0]['awayTeam']

{'teamId': 1610612742,
 'teamName': 'Mavericks',
 'teamCity': 'Dallas',
 'teamTricode': 'DAL',
 'teamSlug': 'mavericks',
 'wins': 0,
 'losses': 1,
 'score': 99,
 'seed': 0}

In [89]:
def is_primitive(value):
    return isinstance(value, (int, float, str, bool))

# Get keys with primitive values

for game_day in data:
    game_day = game_day['games']
    for scheduled_game in game_day:
        game_metadata = {key:value for key, value in scheduled_game.items() if is_primitive(value)}
        game_metadata['homeTeamID'] = scheduled_game['homeTeam']['teamId']
        game_metadata['awayTeamID'] = scheduled_game['awayTeam']['teamId']
        game_metadata['homeTeamName'] = scheduled_game['homeTeam']['teamName']
        game_metadata['awayTeamName'] = scheduled_game['awayTeam']['teamName']

        pd.DataFrame(game_metadata,index=[0]).to_sql('stg_nba_schedule', engine, if_exists='append', index=False, method='multi')

In [41]:
# iterate through the game dates in a season
for game_date in data:
    for game in game_date:
        
    print(game_date['gameDate'])
# iterate through games on date
data[1]['games'][1]

{'gameId': '0012300003',
 'gameCode': '20231007/LALGSW',
 'gameStatus': 1,
 'gameStatusText': '8:30 pm ET',
 'gameSequence': 2,
 'gameDateEst': '2023-10-07T00:00:00Z',
 'gameTimeEst': '1900-01-01T20:30:00Z',
 'gameDateTimeEst': '2023-10-07T20:30:00Z',
 'gameDateUTC': '2023-10-07T04:00:00Z',
 'gameTimeUTC': '1900-01-02T01:30:00Z',
 'gameDateTimeUTC': '2023-10-08T00:30:00Z',
 'awayTeamTime': '2023-10-07T17:30:00Z',
 'homeTeamTime': '2023-10-07T17:30:00Z',
 'day': 'Sat',
 'monthNum': 10,
 'weekNumber': 0,
 'weekName': '',
 'ifNecessary': False,
 'seriesGameNumber': '',
 'seriesText': 'Preseason',
 'arenaName': 'Chase Center',
 'arenaState': 'CA',
 'arenaCity': 'San Francisco',
 'postponedStatus': 'A',
 'branchLink': '',
 'gameSubtype': '',
 'broadcasters': {'nationalBroadcasters': [{'broadcasterScope': 'natl',
    'broadcasterMedia': 'tv',
    'broadcasterId': 7,
    'broadcasterDisplay': 'NBA TV',
    'broadcasterAbbreviation': 'NBA TV',
    'tapeDelayComments': '',
    'broadcasterVideo

In [42]:
flat_dict = {key:value for d in (data[3]['games'][0]['homeTeam'],data[3]['games'][0]['awayTeam']) for key,value in d.items()}

In [43]:
flat_dict

{'teamId': 1610612738,
 'teamName': 'Celtics',
 'teamCity': 'Boston',
 'teamTricode': 'BOS',
 'teamSlug': 'celtics',
 'wins': 0,
 'losses': 0,
 'score': 0,
 'seed': 0}

In [84]:
query = """select
           *
           ,case when city='OPPONENTS' then 0 else 1 end as HOME_FLAG
           ,case when city='OPPONENTS' then 1 else 0 end as AWAY_FLAG
           ,row_number() over (partition by team_id,season order by game_date asc) as TOT_GAMES_PLAYED
           from stg_game_stats"""
df = pd.read_sql_query(query,engine)

In [85]:
import pandas as pd
import numpy as np

def getTotalWinPctg(gameDF):
    gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True) # check
    gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
    gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
    return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)

def getHomeWinPctg(gameDF):
    gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
    gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
    gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
    return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)

def getAwayWinPctg(gameDF):
    gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
    gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
    gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
    return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

def getRollingOE(gameDF):
    gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

def getRollingScoringMargin(gameDF):
    gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

def getRestDays(gameDF):
    gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['GAME_DATE'].shift(1)
    gameDF['NUM_REST_DAYS'] = (pd.to_datetime(gameDF['GAME_DATE']) - pd.to_datetime(gameDF['LAST_GAME_DATE']))/np.timedelta64(1,'D') 
    return gameDF.drop('LAST_GAME_DATE',axis=1)

gameLogs = df
gameLogs = getHomeWinPctg(gameLogs)
gameLogs = getAwayWinPctg(gameLogs)
gameLogs = getTotalWinPctg(gameLogs)
getRollingScoringMargin(gameLogs)
getRollingOE(gameLogs)
gameLogs = getRestDays(gameLogs)

In [113]:
def getGameLogFeatureSet(gameDF):

    def shiftGameLogRecords(gameDF):
        gameDF['LAST_GAME_OE'] = gameLogs.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(1)
        gameDF['LAST_GAME_HOME_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_AWAY_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_TOTAL_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_ROLLING_SCORING_MARGIN'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(1)
        gameDF['LAST_GAME_ROLLING_OE'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(1)
    
    
    skip_columns = ['GAME_ID','SEASON','GAME_DATE']
    
    def getHomeTeamFrame(gameDF):
        homeTeamFrame = gameDF[gameDF['CITY'] != 'OPPONENTS']
        homeTeamFrame = homeTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','W','TEAM_ID','GAME_ID','SEASON','GAME_DATE']]

        colRenameDict = {}
        for col in homeTeamFrame.columns:
            if col not in skip_columns:
                colRenameDict[col] = 'HOME_' + col 

        homeTeamFrame.rename(columns=colRenameDict,inplace=True)

        return homeTeamFrame

    def getAwayTeamFrame(gameDF):
        awayTeamFrame = gameDF[gameDF['CITY'] == 'OPPONENTS']
        awayTeamFrame = awayTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','TEAM_ID','GAME_ID','SEASON']]

        colRenameDict = {}
        for col in awayTeamFrame.columns:
            if col not in skip_columns:
                colRenameDict[col] = 'AWAY_' + col 

        awayTeamFrame.rename(columns=colRenameDict,inplace=True)

        return awayTeamFrame
    
    shiftGameLogRecords(gameLogs)
    awayTeamFrame = getAwayTeamFrame(gameLogs)
    homeTeamFrame = getHomeTeamFrame(gameLogs)
    
    return pd.merge(homeTeamFrame, awayTeamFrame, how="inner", on=[ "GAME_ID","SEASON"])


df = getGameLogFeatureSet(gameLogs)

In [162]:
tot_rows = len(gameLogs)

for i in range(0,tot_rows,5):
    chunk = gameLogs.iloc[i:i + 5]
    chunk.to_sql('wrk_wl_features', engine, if_exists='append', index=False, method='multi')

In [114]:
df.columns

Index(['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_NUM_REST_DAYS', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_TOTAL_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'HOME_W', 'HOME_TEAM_ID', 'GAME_ID', 'SEASON', 'GAME_DATE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_NUM_REST_DAYS', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_TOTAL_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE',
       'AWAY_TEAM_ID'],
      dtype='object')

In [118]:
tot_rows = len(df)

for i in range(0,tot_rows,5):
    chunk = df.iloc[i:i + 5]
    chunk.to_sql('fct_wl_features', engine, if_exists='append', index=False, method='multi')

In [207]:
import pandas as pd 
import requests

url = "https://nba-prediction-api.onrender.com/all"
response = requests.get(url)
df = pd.read_json(response.json(),orient='records')
df

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
0,0.535088,NaN,2.0,1.000000,1.000000,15.000000,0.549056,1,1610612737,22000041,2020-21,2020-12-28,0.470588,0.000000,2.0,0.000000,0.000000,-9.500000,0.519910,1610612765
1,0.567797,1.000000,1.0,0.750000,0.800000,7.333333,0.592552,0,1610612737,22000083,2020-21,2021-01-02,0.531532,0.666667,2.0,0.500000,0.600000,-1.666667,0.549664,1610612739
2,0.500000,0.500000,2.0,0.750000,0.666667,3.000000,0.559789,0,1610612737,22000095,2020-21,2021-01-04,0.580357,0.500000,2.0,0.500000,0.500000,-1.333333,0.510969,1610612752
3,0.561404,0.333333,2.0,0.750000,0.571429,2.666667,0.543067,0,1610612737,22000110,2020-21,2021-01-06,0.528000,0.333333,2.0,0.250000,0.285714,-15.666667,0.533390,1610612766
4,0.531532,0.250000,2.0,0.600000,0.444444,-7.000000,0.530978,1,1610612737,22000153,2020-21,2021-01-11,0.536585,0.833333,2.0,0.500000,0.700000,-6.666667,0.559370,1610612755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.609756,0.333333,2.0,0.307692,0.320000,-1.666667,0.566357,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
3536,0.656489,0.351351,3.0,0.325000,0.337662,5.666667,0.613386,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
3537,0.508197,0.342105,2.0,0.325000,0.333333,-7.000000,0.579533,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
3538,0.571429,0.333333,2.0,0.325000,0.329114,-15.666667,0.578705,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [208]:
# retreive max record 
# 
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
away_max_dates = df.groupby(['AWAY_TEAM_ID'])['GAME_DATE'].idxmax()
away_team_df = df.iloc[away_max_dates].reset_index(drop=True).rename(columns={'AWAY_TEAM_ID': 'TEAM_ID'})
away_team_df = away_team_df[['TEAM_ID','GAME_ID','GAME_DATE']]

home_max_dates = df.groupby(['HOME_TEAM_ID'])['GAME_DATE'].idxmax()
home_team_df = df.iloc[home_max_dates].reset_index(drop=True).rename(columns={'HOME_TEAM_ID': 'TEAM_ID'})
home_team_df = home_team_df[['TEAM_ID','GAME_ID','GAME_DATE']]

combined_df = pd.concat([home_team_df,away_team_df],ignore_index=True)
team_last_game = combined_df.iloc[combined_df.groupby(['TEAM_ID'])['GAME_DATE'].idxmax()].reset_index(drop=True)



In [219]:
df.iloc[:,-13:]

,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
0,1,1610612737,22000041,2020-21,2020-12-28,0.470588,0.000000,2.0,0.000000,0.000000,-9.500000,0.519910,1610612765
1,0,1610612737,22000083,2020-21,2021-01-02,0.531532,0.666667,2.0,0.500000,0.600000,-1.666667,0.549664,1610612739
2,0,1610612737,22000095,2020-21,2021-01-04,0.580357,0.500000,2.0,0.500000,0.500000,-1.333333,0.510969,1610612752
3,0,1610612737,22000110,2020-21,2021-01-06,0.528000,0.333333,2.0,0.250000,0.285714,-15.666667,0.533390,1610612766
4,1,1610612737,22000153,2020-21,2021-01-11,0.536585,0.833333,2.0,0.500000,0.700000,-6.666667,0.559370,1610612755
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
3536,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
3537,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
3538,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [220]:
latest_home_game = df.iloc[:,:12].merge(team_last_game, left_on=['HOME_TEAM_ID','GAME_DATE','GAME_ID'], right_on=['TEAM_ID','GAME_DATE','GAME_ID'])
latest_away_game = df.iloc[:,-13:].merge(team_last_game, left_on=['AWAY_TEAM_ID','GAME_DATE','GAME_ID'], right_on=['TEAM_ID','GAME_DATE','GAME_ID']).drop('HOME_TEAM_ID',axis=1)

In [236]:
col_rename = {col:col[5:] if col[:5] in ['HOME_'] else col for col in latest_home_game.columns}
latest_home_game = latest_home_game.drop(['HOME_TEAM_ID','HOME_W','SEASON'],axis=1).rename(columns=col_rename)[['TEAM_ID','GAME_ID','GAME_DATE','LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG', 'NUM_REST_DAYS', 'LAST_GAME_AWAY_WIN_PCTG',
       'LAST_GAME_TOTAL_WIN_PCTG', 'LAST_GAME_ROLLING_SCORING_MARGIN',
       'LAST_GAME_ROLLING_OE']]

In [237]:
#latest_away_game.iloc[:,-10:].columns[3][5:]
col_rename = {col:col[5:] if col[:5] == 'AWAY_' else col for col in latest_away_game.columns}
latest_away_game = latest_away_game.iloc[:,-12:].drop('AWAY_TEAM_ID',axis=1).rename(columns=col_rename)[['TEAM_ID','GAME_ID','GAME_DATE','LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG', 'NUM_REST_DAYS', 'LAST_GAME_AWAY_WIN_PCTG',
       'LAST_GAME_TOTAL_WIN_PCTG', 'LAST_GAME_ROLLING_SCORING_MARGIN',
       'LAST_GAME_ROLLING_OE']]


In [241]:
pd.concat([latest_away_game,latest_home_game]).reset_index(drop=True).drop_duplicates()

,TEAM_ID,GAME_ID,GAME_DATE,LAST_GAME_OE,LAST_GAME_HOME_WIN_PCTG,NUM_REST_DAYS,LAST_GAME_AWAY_WIN_PCTG,LAST_GAME_TOTAL_WIN_PCTG,LAST_GAME_ROLLING_SCORING_MARGIN,LAST_GAME_ROLLING_OE
0,1610612737,22201216,2023-04-09,0.588652,0.585366,2.0,0.425000,0.506173,10.333333,0.602885
1,1610612766,22201218,2023-04-09,0.500000,0.317073,2.0,0.325000,0.320988,-14.333333,0.538177
2,1610612765,22201223,2023-04-09,0.581197,0.219512,2.0,0.200000,0.209877,-7.000000,0.575409
3,1610612759,22201224,2023-04-09,0.636364,0.341463,1.0,0.175000,0.259259,-13.000000,0.573532
4,1610612758,22201227,2023-04-09,0.534483,0.560976,2.0,0.625000,0.592593,-2.666667,0.597749
5,1610612762,22201228,2023-04-09,0.573643,0.560976,1.0,0.350000,0.456790,-4.666667,0.572762
6,1610612753,22201219,2023-04-09,0.436364,0.487805,2.0,0.350000,0.419753,-15.000000,0.503979
7,1610612740,22201225,2023-04-09,0.605263,0.658537,2.0,0.375000,0.518519,-1.000000,0.577084
8,1610612755,22201217,2023-04-09,0.591549,0.707317,2.0,0.600000,0.654321,-7.000000,0.570550
9,1610612754,22201220,2023-04-09,0.573770,0.487805,2.0,0.350000,0.419753,-8.666667,0.568709


In [27]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

df = df.dropna()
X = df[df['SEASON'] != '2022-23'].drop(['SEASON','HOME_W'],axis=1)
y = df[df['SEASON'] != '2022-23']['HOME_W']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.33)

In [31]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_data_train = scaler.transform(X_train)

scaler.fit(X_test)
scaled_data_test = scaler.transform(X_test)

model = LogisticRegression()
model.fit(scaled_data_train,y_train)
model.score(scaled_data_test,y_test)

0.603542234332425

In [33]:
import joblib
joblib.dump(model,'NBAHomeTeamWinLoss.pkl')

['NBAHomeTeamWinLoss.pkl']

In [34]:
model = joblib.load('NBAHomeTeamWinLoss.pkl')
model.score(scaled_data_train,y_train)

0.6393552719946273

- New Script Requirements
    - Get Team/Player tables
    - Update historic games table
    - Update most recent features

In [38]:
from nba_api.stats.static import teams,players 
import pandas as pd

df = pd.DataFrame(teams.get_teams())
df.rename(columns={'id':'team_id'},inplace=True)
df


,team_id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [40]:
pd.DataFrame(players.get_players())

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4810,1627790,Ante Zizic,Ante,Zizic,False
4811,78647,Jim Zoet,Jim,Zoet,False
4812,78648,Bill Zopf,Bill,Zopf,False
4813,1627826,Ivica Zubac,Ivica,Zubac,True


In [46]:
import datetime 

datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S.%f")

'2023-10-08 23:45:15.827183'

In [79]:
import datetime 

sql_query = f"""SELECT cast(gameId as signed) as GAME_ID
                     ,gameDateTimeUTC
                     ,seriesText
                     ,homeTeamID
                     ,awayTeamID
                     ,homeTeamName
                     ,awayTeamName
                     ,postponedStatus
                FROM 
                    stg_nba_schedule  """

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
df

,GAME_ID,gameDateTimeUTC,seriesText,homeTeamID,awayTeamID,homeTeamName,awayTeamName,postponedStatus
0,12300001,2023-10-05T16:00:00Z,Preseason,1610612750,1610612742,Timberwolves,Mavericks,A
1,12300002,2023-10-07T16:00:00Z,Preseason,1610612742,1610612750,Mavericks,Timberwolves,A
2,12300003,2023-10-08T00:30:00Z,Preseason,1610612744,1610612747,Warriors,Lakers,A
3,12300004,2023-10-08T17:00:00Z,Preseason,1610612749,1610612741,Bucks,Bulls,A
4,12300005,2023-10-08T19:00:00Z,Preseason,1610612765,1610612756,Pistons,Suns,A
...,...,...,...,...,...,...,...,...
1299,22301196,2024-04-14T19:30:00Z,,1610612760,1610612742,Thunder,Mavericks,A
1300,22301197,2024-04-14T19:30:00Z,,1610612759,1610612765,Spurs,Pistons,A
1301,22301198,2024-04-14T19:30:00Z,,1610612744,1610612762,Warriors,Jazz,A
1302,22301199,2024-04-14T19:30:00Z,,1610612746,1610612745,Clippers,Rockets,A


In [80]:
if [1,2,3] not in [3,4,5]:
    print('not in')

not in


In [153]:
import requests 

url = "https://nba-prediction-api.onrender.com/upcoming_games"
response = requests.get(url)
upcoming_games = pd.read_json(response.json(),orient='records')

In [156]:
upcoming_games.columns

Index(['GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'GAME_DATETIME',
       'HOME_TEAM_ID', 'AWAY_TEAM_ID'],
      dtype='object')

In [157]:
home_team_df.columns[]

Index(['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_NUM_REST_DAYS', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_TOTAL_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'HOME_W', 'HOME_TEAM_ID', 'GAME_ID', 'SEASON', 'GAME_DATE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_NUM_REST_DAYS', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_TOTAL_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE',
       'AWAY_TEAM_ID'],
      dtype='object')

In [158]:
away_team_df.columns

Index(['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_NUM_REST_DAYS', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_TOTAL_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'HOME_W', 'HOME_TEAM_ID', 'GAME_ID', 'SEASON', 'GAME_DATE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_NUM_REST_DAYS', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_TOTAL_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE',
       'AWAY_TEAM_ID'],
      dtype='object')

In [159]:
pd.concat([home_team_df,away_team_df])

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
0,0.613445,0.600000,2.0,0.425000,0.512500,12.666667,0.607685,0,1610612737,22201205,2022-23,2023-04-07,0.548673,0.707317,1.0,0.589744,0.650000,-13.000000,0.558902,1610612755
1,0.650794,0.775000,2.0,0.609756,0.691358,7.000000,0.581005,1,1610612738,22201216,2022-23,2023-04-09,0.588652,0.585366,2.0,0.425000,0.506173,10.333333,0.602885,1610612737
2,0.606299,0.775000,3.0,0.487805,0.629630,12.666667,0.594935,0,1610612739,22201218,2022-23,2023-04-09,0.500000,0.317073,2.0,0.325000,0.320988,-14.333333,0.538177,1610612766
3,0.603053,0.650000,2.0,0.375000,0.512500,-1.000000,0.585330,1,1610612740,22201209,2022-23,2023-04-07,0.651515,0.575000,2.0,0.600000,0.587500,10.666667,0.643380,1610612752
4,0.603448,0.525000,2.0,0.439024,0.481481,-9.333333,0.555028,1,1610612741,22201223,2022-23,2023-04-09,0.581197,0.219512,2.0,0.200000,0.209877,-7.000000,0.575409,1610612765
5,0.546218,0.575000,2.0,0.365854,0.469136,-0.333333,0.575307,0,1610612742,22201224,2022-23,2023-04-09,0.636364,0.341463,1.0,0.175000,0.259259,-13.000000,0.573532,1610612759
6,0.557252,0.825000,1.0,0.463415,0.641975,-9.666667,0.576066,1,1610612743,22201227,2022-23,2023-04-09,0.534483,0.560976,2.0,0.625000,0.592593,-2.666667,0.597749,1610612758
7,0.548387,0.800000,2.0,0.230769,0.518987,8.000000,0.601454,1,1610612744,22201187,2022-23,2023-04-04,0.548077,0.575000,2.0,0.384615,0.481013,-4.333333,0.557743,1610612760
8,0.563025,0.325000,2.0,0.153846,0.240506,-9.333333,0.581120,1,1610612745,22201183,2022-23,2023-04-04,0.566667,0.825000,2.0,0.500000,0.666667,-8.000000,0.538283,1610612743
9,0.584071,0.550000,3.0,0.500000,0.525000,-5.000000,0.558029,1,1610612746,22201215,2022-23,2023-04-08,0.615942,0.425000,2.0,0.400000,0.412500,-3.333333,0.588846,1610612757
